In [1]:
import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
import itertools

In [2]:
inputPath = '../input/newSplited/'
outputPath = '../output/newSplited/'

In [3]:
TOPN_candidate = 100
SETS = [2, 3]
VER = 6
SUBSETNUM = 2

TOPN_b2b = 20
TOPN_cartsOrders = 20
TOPN_clicks = 20
TOPN_c2c = 20
TOPN_click2cart = 20
TOPN_click2order = 20

note = 'covisit_20_20_20_newSuggester2'
outputNote = 'covisit_20_20_20_newSuggester2_Covwgt_timeWgt'

predTypes = ['clicks']
type_labels = {'clicks':0, 'carts':1, 'orders':2}
type2covisitType = {'clicks': 'clicks', 'carts': 'cartsOrders', 'orders': 'cartsOrders'}

In [4]:
def load_df(path):    
    dfs = []
    for e, chunk_file in enumerate(glob.glob(path)):
        chunk = pd.read_parquet(chunk_file)
        chunk.ts = (chunk.ts/1000).astype('int32')
        chunk['type'] = chunk['type'].map(type_labels).astype('int8')
        dfs.append(chunk)
    return pd.concat(dfs).reset_index(drop=True) #.astype({"ts": "datetime64[ms]"})

In [5]:
for SET in SETS:
    coVisitSaveFolder = outputPath + f'coVisit_set{SET}_top{TOPN_b2b}_{TOPN_clicks}_{TOPN_cartsOrders}/'
    candidatesSavePath = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{note}/'
    candidatesSavePath_timeWgt = outputPath + f'candidates_set{SET}_top{TOPN_candidate}_{outputNote}/'
    ! mkdir {candidatesSavePath_timeWgt}
    if SET == 1:
        val_A = load_df('../input/split_2-1_pqt/test_parquets/*')
    elif SET == 2:
        val_A = pd.read_parquet('../input/splited/test.parquet')
    elif SET == 3:
        val_A = load_df('../input/parquets/test_parquets/*')
        # val_A = val_A.drop_duplicates((['session', 'aid']))

    val_A = val_A.sort_values(['session', 'ts'], ascending=[True, False]).reset_index(drop=True)
    val_A['ts'] = val_A.groupby('session').ts.apply(lambda x: np.logspace(1, 0.1, len(x), base=2, endpoint=True) - 1).explode().reset_index(drop=True)

    for k in range(0, 8):
        if k == 0:
            covistWeight_clicks = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_clicks_v{VER}_{k}.pqt')
        else:
            covistWeight_clicks = pd.concat([covistWeight_clicks, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_clicks_v{VER}_{k}.pqt')], axis=0)
    for k in range(0, 8):
        if k == 0:
            covistWeight_cl2cl = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_c2c}_click2click_v{VER}_{k}.pqt')
        else:
            covistWeight_cl2cl = pd.concat([covistWeight_cl2cl, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_c2c}_click2click_v{VER}_{k}.pqt')], axis=0)

    for k in range(0, 8):
        if k == 0:
            covistWeight_cartsOders = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_cartsOrders_v{VER}_{k}.pqt')
        else:
            covistWeight_cartsOders = pd.concat([covistWeight_cartsOders, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_clicks}_cartsOrders_v{VER}_{k}.pqt')], axis=0)
    for k in range(0, 2):
        if k == 0:
            covistWeight_b2b = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_b2b}_buy2buy_v{VER}_{k}.pqt')
        else:
            covistWeight_b2b = pd.concat([covistWeight_b2b, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_b2b}_buy2buy_v{VER}_{k}.pqt')], axis=0)
    for k in range(0, 8):
        if k == 0:
            covistWeightcl2ca = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2cart}_click2cart_v{VER}_{k}.pqt')
        else:
            covistWeightcl2ca = pd.concat([covistWeightcl2ca, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2cart}_click2cart_v{VER}_{k}.pqt')], axis=0)

    for k in range(0, 8):
        if k == 0: 
            covistWeightcl2or = pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2order}_click2order_v{VER}_{k}.pqt')
        else:
            covistWeightcl2or = pd.concat([covistWeightcl2or, pd.read_parquet(coVisitSaveFolder + f'top_{TOPN_click2order}_click2order_v{VER}_{k}.pqt')], axis=0)


    for predType in predTypes:
        print('type:', predType)
        for s in range(SUBSETNUM):     
            print('sub:', s)    
            candidate = pd.read_parquet(candidatesSavePath + f'{predType}Candidate_{s}.pqt')
            candidate['session'] = candidate.session.astype('int32')
            df = val_A.merge(candidate[['session', 'aid']],on='session')
            if predType == 'clicks':
                df = df.merge(covistWeight_clicks, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeight_cl2cl, on=['aid_x', 'aid_y'], how='left').fillna(0)
                cols = ['wgt_x', 'wgt_y']
                for col in cols:
                    df[col] = df[col] * df['ts']
                df.loc[df.type.isin([1, 2]), 'wgt_y'] = 0
                df = df.groupby(['session', 'aid_y']).agg({'wgt_x':'mean', 'wgt_y':'mean'}).reset_index()
                df.columns = ['session', 'aid', 'covWgt_t', 'covWgt_cl2cl_t']
            elif predType == 'carts':
                df = df.merge(covistWeight_cartsOders, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeight_b2b, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeightcl2ca, on=['aid_x', 'aid_y'], how='left').fillna(0)
                cols = ['wgt_x', 'wgt_y', 'wgt']
                for col in cols:
                    df[col] = df[col] * df['ts']
                df.loc[df.type == 0, 'wgt_y'] = 0
                df.loc[df.type.isin([1, 2]), 'wgt'] = 0
                df = df.groupby(['session', 'aid_y']).agg({'wgt_x':'mean', 'wgt_y':'mean', 'wgt':'mean'}).reset_index()
                df.columns = ['session', 'aid', 'covWgt_t', 'covWgt_b2b_t', 'covWgt_cl2cr_t']
            else:
                df = df.merge(covistWeight_cartsOders, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeight_b2b, on=['aid_x', 'aid_y'], how='left').fillna(0)
                df = df.merge(covistWeightcl2or, on=['aid_x', 'aid_y'], how='left').fillna(0)
                cols = ['wgt_x', 'wgt_y', 'wgt']
                for col in cols:
                    df[col] = df[col] * df['ts']
                df.loc[df.type == 0, 'wgt_y'] = 0
                df.loc[df.type.isin([1, 2]), 'wgt'] = 0
                df = df.groupby(['session', 'aid_y']).agg({'wgt_x':'mean', 'wgt_y':'mean', 'wgt':'mean'}).reset_index()
                df.columns = ['session', 'aid', 'covWgt_t', 'covWgt_b2b_t', 'covWgt_cl2or_t']
                
            candidate = candidate.merge(df, on=['session', 'aid'], how='left')
            candidate.to_parquet(candidatesSavePath_timeWgt + f'{predType}Candidate_{s}.pqt')

mkdir: cannot create directory ‘../output/newSplited/candidates_set2_top100_covisit_20_20_20_newSuggester2_Covwgt_timeWgt/’: File exists
type: clicks
sub: 0
sub: 1
mkdir: cannot create directory ‘../output/newSplited/candidates_set3_top100_covisit_20_20_20_newSuggester2_Covwgt_timeWgt/’: File exists
type: clicks
sub: 0
sub: 1


In [8]:
df = pd.read_parquet(candidatesSavePath_timeWgt + f'{predType}Candidate_{s}.pqt')

In [10]:
df[:50]

,session,aid,covWgt_t,covWgt_cl2cl_t
0,13800404,167592,0.000000,4.080795
1,13800404,580923,0.000000,3.514843
2,13800404,871573,0.000000,3.235302
3,13800404,152276,0.000000,2.952581
4,13800404,74628,0.000000,2.928370
5,13800404,94845,0.000000,2.750992
6,13800404,1600099,13.716654,5.413277
7,13800404,352199,11.198678,4.122333
8,13800404,752033,10.110028,4.122333
9,13800404,628921,10.110028,4.070464
